In [ ]:
%pip install chromadb
%pip install ollama
%pip install pypdf

In [ ]:
import os
import chromadb
import ollama # Added for the live LLM call
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pypdf import PdfReader

In [ ]:
# ID of our fine-tuned model on the Hugging Face Hub
EMBEDDING_MODEL_ID = "arvindcreatrix/bge-baes-my-qna-model"

# The specific LLM we want to use for generation via Ollama
GENERATOR_MODEL_ID = "llama2"

# Path to the PDF file you want to ask questions about
PDF_PATH = "/content/sample_explain.pdf"

# ChromaDB settings
CHROMA_PATH = "./rag_chroma_db"
COLLECTION_NAME = "rag_collection"

# The instruction prefix required by the BGE embedding model
INSTRUCTION_PREFIX = "Represent this sentence for searching relevant passages: "

In [ ]:
def get_llm_response(prompt: str) -> str:
    """
    Sends the prompt to a locally running Qwen2 model using Ollama.

    Args:
        prompt: The complete prompt to be sent to the LLM.

    Returns:
        The text response from the LLM.
    """
    try:
        # Send the prompt to the specified model
        response = ollama.chat(
            model=GENERATOR_MODEL_ID,
            messages=[{'role': 'user', 'content': prompt}]
        )
        # Extract and return the content from the response
        return response['message']['content']
    except Exception as e:
        print(f"\nAn error occurred while calling the LLM via Ollama: {e}")
        return "Error: Could not get a response from the language model. Is Ollama running?"


In [ ]:
def load_and_split_pdf(file_path: str) -> list[str]:
    """Loads a PDF, extracts text, and splits it into chunks."""
    print(f"Loading and splitting document: {file_path}")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"PDF file not found at: {file_path}")

    reader = PdfReader(file_path)
    text = "\n".join(page.extract_text() for page in reader.pages)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )

    chunks = text_splitter.split_text(text)
    print(f"Document split into {len(chunks)} chunks.")
    return chunks

def setup_rag_pipeline(pdf_path: str):
    """Sets up the entire RAG pipeline: embedding, chunking, and indexing."""
    print("--- Setting up RAG pipeline ---")

    # 1. Load and split the document
    chunks = load_and_split_pdf(pdf_path)

    # 2. Load the fine-tuned embedding model
    print(f"Loading embedding model: {EMBEDDING_MODEL_ID}")
    embedding_model = SentenceTransformer(EMBEDDING_MODEL_ID)

    # 3. Setup ChromaDB
    print(f"Setting up ChromaDB at: {CHROMA_PATH}")
    chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
    collection = chroma_client.get_or_create_collection(name=COLLECTION_NAME)

    # 4. Embed and index the chunks
    print(f"Embedding and indexing {len(chunks)} chunks...")
    chunk_ids = [str(i) for i in range(len(chunks))]
    collection.add(
        ids=chunk_ids,
        documents=chunks,
        embeddings=embedding_model.encode(chunks).tolist()
    )

    print("--- RAG pipeline setup complete! ---")
    return collection, embedding_model

def ask_question(question: str, collection, embedding_model):
    """Handles the retrieval and generation for a single question."""

    # 1. Embed the user's question
    instructed_query = INSTRUCTION_PREFIX + question
    query_embedding = embedding_model.encode(instructed_query).tolist()

    # 2. Retrieve relevant context from ChromaDB
    retrieved_results = collection.query(
        query_embeddings=[query_embedding],
        n_results=3
    )

    retrieved_context = "\n\n---\n\n".join(retrieved_results['documents'][0])

    # 3. Construct the prompt for the LLM
    prompt = f"""
    Context:
    {retrieved_context}

    Question: {question}

    Based on the context provided, please answer the question. If the context does not contain the answer, state that.
    Answer:
    """

    # 4. Get the response from the LLM
    answer = get_llm_response(prompt)
    return answer

In [ ]:
if __name__ == "__main__":
    # Create a dummy PDF for demonstration if one doesn't exist
    if not os.path.exists(PDF_PATH):
        print(f"Creating a dummy PDF at: {PDF_PATH}")
        try:
            from fpdf import FPDF
            pdf = FPDF()
            pdf.add_page()
            pdf.set_font("Arial", size=12)
            pdf.multi_cell(0, 10,
                "The theory of general relativity was proposed by Albert Einstein. It describes gravity as a fundamental property of spacetime. "
                "Isaac Newton developed the laws of motion and universal gravitation, which were the dominant theories for centuries before Einstein. "
                "In biology, the mitochondrion is known as the powerhouse of the cell, responsible for generating most of the cell's supply of adenosine triphosphate (ATP)."
            )
            pdf.output(PDF_PATH)
        except ImportError:
            print("Please install `fpdf` (pip install fpdf) to create the dummy PDF.")
            exit()

    # Setup the RAG pipeline (this will take a moment)
    collection, embedding_model = setup_rag_pipeline(PDF_PATH)

    # Start the interactive Q&A loop
    print("\n--- Ollama and Qwen2-0.5B Ready! ---")
    print("Enter your questions. Type 'exit' to quit.")
    while True:
        user_question = input("\nYour Question: ")
        if user_question.lower() == 'exit':
            break

        answer = ask_question(user_question, collection, embedding_model)
        print("\nAnswer:")
        print(answer)

    print("\n--- Session Ended ---")

In [ ]:
if __name__ == "__main__":
    # Create a dummy PDF for demonstration if one doesn't exist
    if not os.path.exists(PDF_PATH):
        print(f"Creating a dummy PDF at: {PDF_PATH}")
        try:
            from fpdf import FPDF
            pdf = FPDF()
            pdf.add_page()
            pdf.set_font("Arial", size=12)
            pdf.multi_cell(0, 10,
                "The theory of general relativity was proposed by Albert Einstein. It describes gravity as a fundamental property of spacetime. "
                "Isaac Newton developed the laws of motion and universal gravitation, which were the dominant theories for centuries before Einstein. "
                "In biology, the mitochondrion is known as the powerhouse of the cell, responsible for generating most of the cell's supply of adenosine triphosphate (ATP)."
            )
            pdf.output(PDF_PATH)
        except ImportError:
            print("Please install `fpdf` (pip install fpdf) to create the dummy PDF.")
            exit()

    # Setup the RAG pipeline (this will take a moment)
    collection, embedding_model = setup_rag_pipeline(PDF_PATH)

    # Start the interactive Q&A loop
    print("\n--- Ollama and Qwen2-0.5B Ready! ---")
    print("Enter your questions. Type 'exit' to quit.")
    while True:
        user_question = input("\nYour Question: ")
        if user_question.lower() == 'exit':
            break

        answer = ask_question(user_question, collection, embedding_model)
        print("\nAnswer:")
        print(answer)

    print("\n--- Session Ended ---")

In [ ]:
# ID of our fine-tuned model on the Hugging Face Hub
EMBEDDING_MODEL_ID = "BAAI/bge-base-en-v1.5"

# The specific LLM we want to use for generation via Ollama
GENERATOR_MODEL_ID = "llama2"

# Path to the PDF file you want to ask questions about
PDF_PATH = "/content/sample_explain.pdf"

# ChromaDB settings
CHROMA_PATH = "./rag_chroma_db"
COLLECTION_NAME = "rag_collection"

# The instruction prefix required by the BGE embedding model
INSTRUCTION_PREFIX = "Represent this sentence for searching relevant passages: "

In [ ]:
if __name__ == "__main__":
    # Create a dummy PDF for demonstration if one doesn't exist
    if not os.path.exists(PDF_PATH):
        print(f"Creating a dummy PDF at: {PDF_PATH}")
        try:
            from fpdf import FPDF
            pdf = FPDF()
            pdf.add_page()
            pdf.set_font("Arial", size=12)
            pdf.multi_cell(0, 10,
                "The theory of general relativity was proposed by Albert Einstein. It describes gravity as a fundamental property of spacetime. "
                "Isaac Newton developed the laws of motion and universal gravitation, which were the dominant theories for centuries before Einstein. "
                "In biology, the mitochondrion is known as the powerhouse of the cell, responsible for generating most of the cell's supply of adenosine triphosphate (ATP)."
            )
            pdf.output(PDF_PATH)
        except ImportError:
            print("Please install `fpdf` (pip install fpdf) to create the dummy PDF.")
            exit()

    # Setup the RAG pipeline (this will take a moment)
    collection, embedding_model = setup_rag_pipeline(PDF_PATH)

    # Start the interactive Q&A loop
    print("\n--- Ollama and Qwen2-0.5B Ready! ---")
    print("Enter your questions. Type 'exit' to quit.")
    while True:
        user_question = input("\nYour Question: ")
        if user_question.lower() == 'exit':
            break

        answer = ask_question(user_question, collection, embedding_model)
        print("\nAnswer:")
        print(answer)

    print("\n--- Session Ended ---")